In [ ]:
%%markdown

완전한 에이전트 루프를 갖춘 Movie Agent를 구축하세요!
에이전트가 수행해야 할 작업:
영화에 대한 사용자 질문을 받습니다.
어떤 도구를 호출할지 결정합니다 (필요한 경우).
실제로 API를 호출하여 LLM에게 실제 결과를 반환합니다.
LLM이 최종 답변을 줄 때까지 루프를 계속합니다.

## API 참고
기본 URL: https://nomad-movies.nomadcoders.workers.dev

| 엔드포인트 | 설명 |
|---|---|
| `/movies` | 인기 영화 목록 |
| `/movies/:id` | ID로 영화 상세 정보 조회 |
| `/movies/:id/credits` | 영화의 출연진 및 제작진 조회 |
| `/movies/:id/similar` | 유사한 영화 조회 |

## 요구사항
실제 API를 호출하는 최소 3개의 도구를 갖춘 에이전트를 만드세요:
get_popular_movies() - /movies 호출
get_movie_details(id) - /movies/:id 호출
get_similar_movies(id) - /movies/:id/similar 호출

에이전트가 갖춰야 할 조건:
(수동 프롬프팅이 아닌) OpenAI tools 파라미터를 사용하세요.
응답에서 tool_calls를 처리하세요.
실제 API를 호출하고 결과를 다시 전달해야 합니다.
메모리를 활용한 멀티턴 대화를 지원해야 합니다.

## 예시 상호작용
User: 지금 인기 있는 영화 알려줘
Agent: [get_popular_movies() 호출]
Agent: 현재 인기 영화 목록입니다: 1. 듄: 파트 2 (ID: 693134)...

User: 듄에 대해 더 알려줘
Agent: [get_movie_details(693134) 호출]
Agent: 듄: 파트 2는 드니 빌뇌브 감독의 2024년 SF 대작으로...

User: 비슷한 영화 추천해 줄래?
Agent: [get_similar_movies(693134) 호출]
Agent: 듄을 좋아하셨다면 이런 영화를 추천드립니다: 블레이드 러너 2049, 어라이벌...



완전한 에이전트 루프를 갖춘 Movie Agent를 구축하세요!
에이전트가 수행해야 할 작업:
영화에 대한 사용자 질문을 받습니다.
어떤 도구를 호출할지 결정합니다 (필요한 경우).
실제로 API를 호출하여 LLM에게 실제 결과를 반환합니다.
LLM이 최종 답변을 줄 때까지 루프를 계속합니다.

## API 참고
기본 URL: https://nomad-movies.nomadcoders.workers.dev

| 엔드포인트 | 설명 |
|---|---|
| `/movies` | 인기 영화 목록 |
| `/movies/:id` | ID로 영화 상세 정보 조회 |
| `/movies/:id/credits` | 영화의 출연진 및 제작진 조회 |
| `/movies/:id/similar` | 유사한 영화 조회 |

## 요구사항
실제 API를 호출하는 최소 3개의 도구를 갖춘 에이전트를 만드세요:
get_popular_movies() - /movies 호출
get_movie_details(id) - /movies/:id 호출
get_similar_movies(id) - /movies/:id/similar 호출
에이전트가 갖춰야 할 조건:
(수동 프롬프팅이 아닌) OpenAI tools 파라미터를 사용하세요.
응답에서 tool_calls를 처리하세요.
실제 API를 호출하고 결과를 다시 전달해야 합니다.
메모리를 활용한 멀티턴 대화를 지원해야 합니다.

## 예시 상호작용
User: 지금 인기 있는 영화 알려줘
Agent: [get_popular_movies() 호출]
Agent: 현재 인기 영화 목록입니다: 1. 듄: 파트 2 (ID: 693134)...

User: 듄에 대해 더 알려줘
Agent: [get_movie_details(693134) 호출]
Agent: 듄: 파트 2는 드니 빌뇌브 감독의 2024년 SF 대작으로...

User: 비슷한 영화 추천해 줄래?
Agent: [get_similar_movies(693134) 호출]
Agent: 듄을 좋아하셨다면 이런 영화를 추천드립니다: 블레이드 러너 2049, 어라이벌...


In [31]:
import openai, json
import requests

URL = "https://nomad-movies.nomadcoders.workers.dev"
client = openai.OpenAI()
messages = []

def get_popular_movies():
    response = requests.get(f"{URL}/movies")
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}"

def get_movie_details(id):
    response = requests.get(f"{URL}/movies/{id}")
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}"

def get_similar_movies(id):
    response = requests.get(f"{URL}/movies/{id}/similar")
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}"


FUNCTION_MAP = {
    "get_popular_movies": get_popular_movies,
    "get_movie_details": get_movie_details,
    "get_similar_movies": get_similar_movies
}

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_popular_movies",
            "description": "Fetches a list of popular movies.",
            "parameters": {
                "type": "object",
                "properties": {},
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_details",
            "description": "Fetches details of a movie by its ID.",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "string",
                        "description": "The ID of the movie to get details for.",
                    }
                },
                "required": ["id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_similar_movies",
            "description": "Fetches a list of movies similar to the given movie ID.",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "string",
                        "description": "The ID of the movie to find similar movies for.",
                    }
                },
                "required": ["id"],
            }
        },
    }
]

from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            arg_str = ", ".join(str(v) for v in arguments.values()) if arguments else ""
            print(f"Agent: [{function_name}({arg_str}) 호출]")

            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": json.dumps(result) if not isinstance(result, str) else result,
                }
            )

        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"Agent: {message.content}")


def call_ai():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)

In [32]:
while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: 지금 인기 있는 영화 알려줘
Agent: [get_popular_movies() 호출]
Agent: 현재 인기 있는 영화 목록은 다음과 같습니다:

1. **Shelter**
   - 개요: 외딴 섬에서 자활 중인 남자가 폭풍으로부터 어린 소녀를 구출하면서 과거의 적들로부터 그녀를 보호해야 하는 사건이 펼쳐진다.
   - 개봉일: 2026년 1월 28일
   - 평점: 6.9
   - ![포스터](https://image.tmdb.org/t/p/w780/buPFnHZ3xQy6vZEHxbHgL1Pc6CR.jpg)

2. **Mercy**
   - 개요: 미래의 한 형사가 아내를 살해한 혐의로 재판을 받게 되고, AI 재판관에게 자신의 무죄를 입증해야 하는 90분의 시간이 주어진다.
   - 개봉일: 2026년 1월 20일
   - 평점: 7.1
   - ![포스터](https://image.tmdb.org/t/p/w780/pyok1kZJCfyuFapYXzHcy7BLlQa.jpg)

3. **The Bluff**
   - 개요: 평화로운 섬 생활이 복수심에 불타는 전 선장과의 재회로 깨지면서, 기술이 뛰어난 전 해적이 과거의 유혈을 맞딱뜨린다.
   - 개봉일: 2026년 2월 17일
   - 평점: 6.1
   - ![포스터](https://image.tmdb.org/t/p/w780/sojEzvfxR2DBcDSJyAisX8TWjov.jpg)

4. **The Orphans**
   - 개요: 고아 출신 친구 두 명이 사랑하는 이를 잃고, 그녀의 딸이 복수를 위해 나서게 되면서 이야기가 전개된다.
   - 개봉일: 2025년 8월 20일
   - 평점: 6.2
   - ![포스터](https://image.tmdb.org/t/p/w780/hP7mjZr2SVfjAorlRHTdV1XZmHY.jpg)

5. **Hellfire**
   - 개요: 신비로운 과거를 가진 방랑자가 작은 마을에 도착해 범죄 보스의 지배를 깨기 위해 돕는 이야기.
   - 개봉일: 